In [ ]:
import pandas as pd
import bibtexparser
import datetime

#### Set variables you will need

# Set path to a .bib file that has only the papers you are a co-author on
bibtexfile = "/Users/siegfried/Documents/bibtex/mrs_pubs.bib"

# Set how your name appears in your papers, so you do not appear in your own COA
# Note: as you probably an tell, you can add whatever "other" authors you want to get out
#       your COA -- in this case 'the SALSA Science Team' is on my bibliography, so I don't want that
throw_out = ['Matthew R. Siegfried', 'the SALSA Science Team']

# Set the path to your previous COA Table 4 (ideally generated from this code from which 
# it will pull name-institution information for merging with your bibliography. If you are running this
# for the first time, you can just generate a new .xlsx of your existing Table 4 (or make a new table)
# with the note it **must** include columns named 'name' and 'affiliation'.
previous_coa = 'coa_table4_20240129.xlsx'

# Set a path to a list of your collaborators on funded awards and graduate advising. Out of laziness
# I didn't make any interactivity to choose columns, so you must have columns 
# named 'Name', 'Affiliation', and 'End Year'. End Year is the End Year of the collaboration (it can
# be in the future; the code will change any future dates to 1/1/[thisyear]
awards = 'award_collabs.xlsx'

## helper functions
def last_name_first(s):
    '''
    Function to take a name stylized as '[fist name] [MI] [last name]' and make 
    it '[last name], [first] [MI]. This will recognize if someone has a multi-part
    last name in bibtex format (i.e., set off in curly braces) and act appropriately
    '''
    if '{' in s:
        start = s.index('{')
        end = s.index('}')
        lname = s[start+1:end]
        rest = s[0:start]
        out = lname + ', ' + rest
    else:
        str_list = s.split()
        out = str_list[-1] + ', ' + ' '.join(str_list[0:-1])
    return out

In [ ]:
# Make filename string as 'coa_table4_[YYYYMMDD].xlsx
today = datetime.date.today()
datestr = '{:04}'.format(today.year) + '{:02}'.format(today.month) + '{:02}'.format(today.day)
coa_out = 'coa_table4_' + datestr + '.xlsx'

# Make the range to look in (last 48 months)
firstyear = today.year - 4

###### WORK ON A: TYPE CONFLICTS
# Open bibtex file
with open(bibtexfile) as bibtex_file:
    library = bibtexparser.load(bibtex_file)
df = pd.DataFrame(library.entries)

# correct for years in or not in {}
for i,row in df.iterrows():
    if len(row['year']) > 4:
        df.loc[i, 'year'] = row.year[1:5]
df['year'] = df['year'].astype(int)

# subset pubs to just time since the first year we want
coa_pubs = df[df.year >= firstyear] 

# make a new dictionary with each person's name as a key and a list of years as values. 
# do some data cleaning while we are dealing with this (e.g., daggers for co-first authors)
coa = dict()

for i,row in coa_pubs.iterrows():
    authors = row.author.split(' and ')
    for a in authors:
        if '$^{\\dagger}$' in a:
            a = a[0:-12]
        if not any(word in a for word in throw_out):
            a = last_name_first(a)
            if a in coa:
                if row.year > coa[a]:
                    coa[a] = row.year
            else:
                coa[a] = row.year

# make a pandas dataframe of A: type conflicts (i.e. co authors)
coa = dict(sorted(coa.items())) # make alphabetical

df_atype = pd.DataFrame(coa.items(), columns=['name','year'])
df_atype['type'] = 'A:'

# grab the old COA file to merge affiliations where we have them
df_oldcoa = pd.read_excel(previous_coa, usecols = ['name', 'affiliation'])
df_atype = df_atype.merge(df_oldcoa.drop_duplicates(subset=['name']), how='left',on='name')

#### WORK ON C: TYPE CONFLICTS (FUNDED AWARD COLLABORATORS AND GRADUATE RESEARCH CO-ADVISORS)
df_ctype = pd.read_excel(awards)
df_ctype = df_ctype.rename(columns = {'Name':'name', 'Affiliation':'affiliation','End Year':'year'})
df_ctype['type'] = 'C:'

# concatenate your type a and type c conflicts into a single pandas dataframe
df_coa = pd.concat([df_atype,df_ctype], ignore_index = True)
df_coa['optional'] = ' ' # add optional column like in the NSF COA spreadsheet

# turn year as integer to a string of 12/31/[year] so that it can copy and paste into the COA spreadsheet
df_coa['year'] = df_coa['year'].astype(str)
for i,row in df_coa.iterrows():
    df_coa.loc[i,'year'] = '12/31/' + str(row.year)

# if it is the current year or higher, 12/31/[year] to 1/1/[thisyear]
df_coa.loc[df_coa['year'].str[-4:].astype(int)>=today.year, 'year'] = '1/1/' + str(today.year)

# and sort alphabetically
df_coa = df_coa.sort_values('name')

# And save
df_coa.to_excel(coa_out, columns=['type','name','affiliation','optional','year'], index=False)